In [45]:
import os
from nltk.corpus import stopwords
import string
import pandas as pd

In [46]:
STOPS = stopwords.words("english")
EXCLUDE = set(string.punctuation) | set(["''", "BR", "--", "/td", "nbsp", "2002", "localhost", "&nbsp"])
SPAMPATH = "./spam/"
HAMPATH = "./easy_ham/"

In [47]:
df = pd.read_csv("./quest2020.csv")
df.head()

,id,interview,interview_score,in_quest,gender,race,credits,gpa,school,honors,all_answers
0,2952,0,NaN,0,m,Asian,47,4.00,Smith,1,My ability to efficiently allocate resources i...
1,3025,0,NaN,0,f,Asian,14,3.67,Smith,1,Volunteering at Terps for Change and now new s...
2,2961,1,3.5,1,f,Asian,15,4.00,Smith,1,Improving as a dancer and making the Maryland ...
3,3041,1,4.0,1,f,Asian,14,3.81,Smith,1,I restructured tennis uniforms for inclusivity...
4,3113,0,NaN,0,m,White,16,3.83,Clark,1,My ability to adapt to different environments....


In [48]:
QUESTWORDS = ['quest', 'QUEST', 'Quest']

def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2))

In [49]:
df['wordcount']=0
df['questcount']=0

# other possibilities:
# word_count
# teamwork_count
# grade_level
# adjective_count
# adverb_count
# sentiment
# quest_count
# quest_upper

for ind in df.index:
    li = list(df.loc[ind, 'all_answers'].split(' '))
    df.loc[ind, 'wordcount'] = len(li)
    df.loc[ind, 'questcount'] = len(intersection(li, QUESTWORDS)) #this is not yet right

df.head()

,id,interview,interview_score,in_quest,gender,race,credits,gpa,school,honors,all_answers,wordcount,questcount
0,2952,0,NaN,0,m,Asian,47,4.00,Smith,1,My ability to efficiently allocate resources i...,1101,1
1,3025,0,NaN,0,f,Asian,14,3.67,Smith,1,Volunteering at Terps for Change and now new s...,890,1
2,2961,1,3.5,1,f,Asian,15,4.00,Smith,1,Improving as a dancer and making the Maryland ...,1185,1
3,3041,1,4.0,1,f,Asian,14,3.81,Smith,1,I restructured tennis uniforms for inclusivity...,927,1
4,3113,0,NaN,0,m,White,16,3.83,Clark,1,My ability to adapt to different environments....,638,1


In [50]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
cv.fit(df['all_answers'])

word_sentence_matrix = cv.transform(df['all_answers'])

sentences_df = pd.DataFrame(word_sentence_matrix.toarray(), columns=cv.get_feature_names())
sentences_df

,000,10,100,1000,108,10g,10k,10th,11,115,...,äúunder,äúwe,äúwhat,äúwhy,äúwomen,äúwrong,äúwushu,äúyou,äúyour,öï
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,0,1,0,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
241,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
242,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
243,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:
X = cv.fit_transform(df['all_answers'])
y = df['in_quest']


In [52]:
from sklearn.linear_model import LogisticRegression

lr_bow_classifier = LogisticRegression()
lr_bow_classifier.fit(X,y)

sorted(list(zip(cv.get_feature_names(), lr_bow_classifier.coef_[0])), key=lambda x: -x[1])

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


[('take', 0.29574549594975075),
 ('we', 0.29085161596854714),
 ('kids', 0.2860312690322726),
 ('individuals', 0.2627125607797531),
 ('her', 0.2084176418283625),
 ('always', 0.20052958229302556),
 ('technology', 0.1963965525863741),
 ('events', 0.1923097540520948),
 ('impact', 0.18875301826252971),
 ('apply', 0.18862604194648452),
 ('either', 0.18799652087499102),
 ('could', 0.18577479839304498),
 ('one', 0.1798487580649352),
 ('see', 0.17970694162409295),
 ('been', 0.17819366739316897),
 ('community', 0.17693546169500285),
 ('success', 0.1744792328972106),
 ('on', 0.17303070169201187),
 ('situations', 0.1725373287367429),
 ('more', 0.17234541627394812),
 ('science', 0.1717396570921182),
 ('overall', 0.16412826317988916),
 ('parents', 0.16091790948655835),
 ('open', 0.158287261364683),
 ('learning', 0.1578236857108163),
 ('as', 0.15660959692970844),
 ('different', 0.1556282097864343),
 ('they', 0.15450427875394143),
 ('provide', 0.15397023576002583),
 ('involved', 0.1533594952313554),
 

In [53]:
lr_bow_classifier.predict(cv.transform(["building cool technology"]))

array([1])